In [1]:
import os
import torch
import torchvision
from torch import nn
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from pathlib import Path
from timeit import default_timer as timer
from torch.utils.data import Subset
import cv2, glob

E:\Major_project\code\deep\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Disease vs Normal

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix

class Bin_EnsembleModel(nn.Module):
    def __init__(self,model1,model2, model3):
        super(Bin_EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
     
        
    def forward(self, x):
        x = x.to(device)
        preds1 = torch.sigmoid(self.model1(x))
        preds2 = torch.sigmoid(self.model2(x))
        preds3 = torch.sigmoid(self.model3(x))
 
        preds = ( preds1 + preds2 + preds3) / 3
        return preds # return the predictions on the same device as input

In [4]:
class Binaryduper_EnsembleModel(nn.Module):
    def __init__(self, model1, model2, model3,model4):
        super(Binaryduper_EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        
    def forward(self, x):
        x = x.to(device)
        preds1 = torch.sigmoid(self.model1(x))
        preds2 = torch.sigmoid(self.model2(x))
        preds3 = torch.sigmoid(self.model3(x))
        preds4 = torch.sigmoid(self.model4(x))
        preds = (preds1 + preds2 + preds3 + preds4) / 4
        return preds # return the predictions on the same device as input

In [5]:
import gradio as gr
import torch
import torchvision.models as models
from torchvision import transforms

model1 = torch.load('E:/Major_project/code/models/Ensemble_Models/Binary_duper4_ensemble_model_02_05.pt',map_location=torch.device('cpu'))
class_names1= ['Disease', 'Normal']
title = "Disease Vs Normal"
description = '''This model classifies the given fundus image into Diseased or Normal '''
input_img = gr.inputs.Image(type='pil', label='Input image')

def preprocess1(image):
    image = image.resize((224,224))
    tensor = transforms.ToTensor()(image)
    tensor = tensor.unsqueeze(0)
    return tensor

def predict1(image):
    tensor = preprocess1(image)
    outputs = model1(tensor)
    _, predicted = torch.max(outputs.data, 1)
    return class_names1[predicted.item()]

interface1 = gr.Interface(
    fn=predict1,
    inputs= input_img,
    outputs=gr.outputs.Label(num_top_classes=2, label='Predicted label'),
    allow_flagging='never',
    title=title,
    description=description
)



E:\Major_project\code\deep\lib\site-packages\gradio\inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
E:\Major_project\code\deep\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
E:\Major_project\code\deep\lib\site-packages\gradio\outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
E:\Major_project\code\deep\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


# Multiple Diseases Vs Normal

In [6]:
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

class Superduper_EnsembleModel(nn.Module):
    def __init__(self, model1, model2, model3,model4):
        super(Superduper_EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        
        
    def forward(self, x):
        x = x.to(device)
        preds1 = torch.sigmoid(self.model1(x))
        preds2 = torch.sigmoid(self.model2(x))
        preds3 = torch.sigmoid(self.model3(x))
        preds4 = torch.sigmoid(self.model4(x))
        preds = (preds1 + preds2 + preds3 + preds4) * 4
        return preds # return the predictions on the same device as input


In [7]:
import gradio as gr
import torch
import torchvision.models as models
from torchvision import transforms

model2 = torch.load('E:/Major_project/code/models/Ensemble_Models/Super_duper4_ensemble_model.pt',map_location=torch.device('cpu'))
# labels= ['Disease', 'Normal']
labels = ['Age_Macular_Degeneration','Cataract','Diabetic_Retinopathy','Glaucoma','Myopia','Healthy Fundus']


def preprocess2(image):
    image = image.resize((224,224))
    tensor = transforms.ToTensor()(image)
    tensor = tensor.unsqueeze(0)
    return tensor
title = "Ocular Disease Recognition"
description = '''This model classifies the given fundus image in different diseases.. '''

def predict2(image):
  image = preprocess2(image)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(model2(image)[0], dim=0)
    confidences = {labels[i]: float(prediction[i]) for i in range(6)}    
  return confidences

interface2 = gr.Interface(
    fn=predict2,
    inputs=gr.inputs.Image(type='pil', label='Input image'),
    outputs=gr.outputs.Label(num_top_classes=4, label='Predicted label'),
    title = title,
    description = description,
    allow_flagging='never'
)

# interface2.launch()

# Diabetes RetinoPathy Stages

In [8]:
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

class EnsembleModel(nn.Module):
    
    def __init__(self, model2, model3, model4):
        super(EnsembleModel, self).__init__()
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        
    def forward(self, x):
        x = x.to(device)
        preds2 = torch.sigmoid(self.model2(x))
        preds3 = torch.sigmoid(self.model3(x))
        preds4 = torch.sigmoid(self.model4(x))

        preds = (preds2 + preds3 + preds4 ) * 27
        return preds # return the predictions on the same device as input

In [9]:
import gradio as gr
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image

model3 = torch.load('E:/Major_project/code/models/Ensemble_Models/dr_ensemble.pt',map_location=torch.device('cpu'))
labels3 = ['Normal','Stage 1','Stage 2','Stage 3','Stage 4']


def preprocess3(image):
    image = np.asarray(image)
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(image_lab)
    clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)
    merged_channels = cv2.merge((cl, a_channel, b_channel))
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    final_image = Image.fromarray(final_image)
    image = final_image.resize((224,224))
    img = transforms.Grayscale(num_output_channels=3)(image)
    tensor = transforms.ToTensor()(img)
    tensor = tensor.unsqueeze(0)
    return tensor

title = "Diabetic Retinopathy Stage Classification"
# description = '''This model classifies the given fundus image in different diseases.. '''

def predict3(image):
  image = preprocess3(image)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(model3(image)[0], dim=0)
    confidences = {labels3[i]: float(prediction[i]) for i in range(5)}    
  return confidences

interface3 = gr.Interface(
    fn=predict3,
    inputs=gr.inputs.Image(type='pil', label='Input image'),
    outputs=gr.outputs.Label(num_top_classes=5, label='Predicted label'),
    allow_flagging='never',
    title=title
#     description=description
)

# interface.launch()

# Gradio Interface

In [10]:
title = "Ocular Disease Recognition"
description = '''This model classifies the given fundus image in different diseases.. '''

interface0 = gr.Parallel(interface1,interface2)
interface = gr.TabbedInterface([interface0, interface3], ["Ocular Diseases", "Diabetic Retinopathy"], title='Ocular Disease Recognition with Ensemble Techniques')
# interface = gr.TabbedInterface([interface1,interface2, interface3], , title='Ocular Disease Recognition with Ensemble Techniques')

interface.launch()

E:\Major_project\code\deep\lib\site-packages\gradio\blocks.py:863: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [11]:
import gradio as gr

print(gr.__version__)

3.27.0
